# Standing Fan EDA

### Speed of fan blades
- It was determined by Android app: Spectroid 
    - https://play.google.com/store/apps/details?id=org.intoorbit.spectrum&hl=en
- Audo settings:
    - Sampling rate: 48 kHz
    - FFT size: 8192 bins (5.9 Hz/bin)
    - Decimations: 5 (0.18 Hz/bin @ DC)
    - Window function: Hann
    - Transform interval: 50 ms
    - Exponential smooting factor: 0.5
- Fan blade speed and rotational speed (3 blades)
    - Speed 1: 57 Hz / 3 = 19 Hz
    - Speed 2: 63 Hz / 3 = 21 Hz
    - Speed 3: 68 Hz / 3 = 22.7 Hz

### Import audio

In [ ]:
import os
from scipy.io import wavfile
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, welch
import seaborn as sb

import sys
sys.path.append('../')
from vibrodiagnostics import pumps, extraction

In [ ]:
PATH = '../datasets/StandingFan'
AUDIO_PATH = os.path.join(PATH, 'AUDIO')
FFT_WINDOW = 2**15

In [ ]:
def plot_psd(
        ax,
        ts: pd.Series,
        fs: int,
        window: int,
        threshold: int = None,
        xlim: int = 300,
        color: bool = True):

    freqs, pxx = extraction.spectral_transform(pd.Series(ts), window, fs)
    if color:
        ax.plot(freqs, pxx, color='darkblue')
        ax.fill_between(freqs, pxx, color='lightblue', alpha=0.3)
    else:
        ax.plot(freqs, pxx)

    peaks = []
    if threshold:
        peaks, _ = find_peaks(pxx, prominence=threshold, distance=10)
        ax.plot(freqs[peaks], pxx[peaks], 'o', color='r')
    ax.set_xlim(0, xlim)

    f = freqs[peaks][0] if len(peaks) > 0 else 0
    return f

In [ ]:
audio = []
filenames = ('1.wav', '2.wav', '3.wav')

for name in filenames:
    name = os.path.join(AUDIO_PATH, name)
    samplerate, data = wavfile.read(name)
    n = data.shape[0]
    duration =  n / samplerate
    channels = data.shape[1] if len(data.shape) > 1 else 1

    audio.append({
        'fs': samplerate,
        'duration': duration,
        'channels': channels,
        'n': n,
        'stream': pd.Series(data)
    })

pd.DataFrame.from_records(audio)

### Plot audio waveforms

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(20, 8))
for i, s in enumerate(audio):
    t = np.linspace(0, s['duration'], s['n'])
    ax[i][0].plot(t, s['stream'])

    x = 15
    d = 0.3
    t = np.linspace(x, x + d, int(s['fs'] * d))
    y = s['stream'][x * s['fs']: int((x + d) * s['fs'])]
    ax[i][1].plot(t, y)

    for j in range(2):
        ax[i][j].set_xlabel('Time [s]')
        ax[i][j].set_ylabel('Amplitude')
        ax[i][j].set_title(f'Speed: {i+1}')

fig.tight_layout()
plt.show()

### Plot audio frequency spectrum
- Results:
    - Speed 1: 19 Hz
    - Speed 2: 21 Hz
    - Speed 3: 22.5 Hz

In [ ]:
window_size = 2 ** 18
blades = 3
fig, ax = plt.subplots(3, 1, figsize=(9, 6))
for i, s in enumerate(audio):
    fundamental = plot_psd(ax[i], s['stream'], s['fs'], window_size, threshold=5000)
    resolution = s['fs'] / window_size
    speed = fundamental / blades
    ax[i].set_title(f'Speed: {i+1}, Rotation: {speed:.4f} +/- {resolution:.4f} Hz')

fig.tight_layout()
plt.show()

#### Open vibration signals from accelerometer

In [ ]:
def load_placement(place: str):
    path = os.path.join(PATH, place.upper())
    filenames = ('1.tsv', '2.tsv', '3.tsv')
    samplerate = pumps.SAMPLING_RATE
    accel = []

    for name in filenames:
        filename = os.path.join(path, name)
        ts = pumps.csv_import(None, filename)
        n = data.shape[0]
        duration =  n / samplerate
        channels = data.shape[1] if len(data.shape) > 1 else 1

        accel.append({
            'speed': name.split('.')[0],
            'fs': samplerate,
            'duration': ts.tail(1).index.to_list()[0],
            'n': len(ts),
            'stream': ts
        })
    return accel

In [ ]:
accel = load_placement('back')   
pd.DataFrame.from_records(accel)

In [ ]:
window_size = 2 ** 18
blades = 3
fig, ax = plt.subplots(3, 1, figsize=(9, 6), sharey=True)
for i, s in enumerate(accel):
    fundamental = plot_psd(ax[i], s['stream']['x'], s['fs'], window_size, 0.12)
    resolution = s['fs'] / window_size
    speed = fundamental / blades
    ax[i].set_title(f'Speed: {i+1}, Rotation: {speed:.4f} +/- {resolution:.4f} Hz')

fig.tight_layout()
plt.show()

In [ ]:
# Show Frequency spectrum
def position_vs_speed(orientation: int, directions: dict, domain: str):
    placements = list(directions.keys())
    window_size = FFT_WINDOW
    fig, ax = plt.subplots(3, 3, figsize=(20, 10))
    data = pd.DataFrame()

    for i, place in enumerate(placements):
        speeds = load_placement(place)
        rotation, axis = list(directions[place][orientation])
        rotation = int(rotation + '1')

        for j, s in enumerate(speeds):
            ts = rotation * s['stream'][axis]
            ts = ts.to_numpy() 
            ts = ts - ts.mean()
            
            ax[j][i].set_title(f'Position: {place}, Speed: {s["speed"]}')
            ax[j][i].set_ylabel('Amplitude [m/s\u00b2]')

            if domain == 'fd':
                plot_psd(ax[j][i], ts, s['fs'], window_size, xlim=500)
                _, ts = extraction.spectral_transform(pd.Series(ts), window_size, s['fs'])
                ax[j][i].set_xlabel('Frequency [Hz]')
                data[place, s['speed']] = ts
        
            elif domain == 'td':
                x = 10
                d = 0.3
                y = ts[x * s['fs']: int((x + d) * s['fs'])]
                t = np.linspace(x, x + d, len(y))
                ax[j][i].plot(t, y)
                ax[j][i].set_xlabel('Time [s]')

    fig.tight_layout()
    plt.show()
    return data

def position_vs_speed_no_expand(orientation: int, directions: dict, xlim: int, ylim: int):
    placements = list(directions.keys())
    window_size = FFT_WINDOW
    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    data = pd.DataFrame()

    for i, place in enumerate(placements):
        speeds = load_placement(place)
        rotation, axis = list(directions[place][orientation])
        rotation = int(rotation + '1')

        for s in speeds:
            ts = rotation * s['stream'][axis]
            ts = ts.to_numpy() 
            ts = ts - ts.mean()
            
            ax[i].set_title(f'Position: {place}')
            ax[i].set_ylabel('Amplitude [m/s\u00b2]')
            ax[i].set_xlabel('Frequency [Hz]')
            ax[i].set_xlim(0, xlim)
            ax[i].set_ylim(0, ylim)

            plot_psd(ax[i], ts, s['fs'], window_size, color=False)

    fig.tight_layout()
    plt.show()

In [ ]:
directions = {
    # tangential, radial, axial
    'back': ['+x', '+y', '+z'],
    'side': ['+x', '-z', '+y'],
    'front': ['+x', '-y', '-z'],
}

In [ ]:
position_vs_speed(0, directions, 'td')

In [ ]:
position_vs_speed(1, directions, 'td')

In [ ]:
position_vs_speed(2, directions, 'td')

In [ ]:
fd_orient = []
fd_orient.append(position_vs_speed(0, directions, 'fd'))

In [ ]:
fd_orient.append(position_vs_speed(1, directions, 'fd'))

In [ ]:
fd_orient.append(position_vs_speed(2, directions, 'fd'))

In [ ]:
position_vs_speed_no_expand(0, directions, xlim=100, ylim=0.10)

In [ ]:
# Heatmap of spectra correlations 
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
for i in range(3):
    sb.heatmap(fd_orient[i].corr(), annot=True, ax=ax[i])
plt.show()

In [ ]:
# Feature range
PARTS = 12
def features_time_domain(root: str, parts: int = PARTS) -> pd.DataFrame:
    frame = pd.DataFrame()
    for filename in extraction.fs_list_files(root):
        df = pumps.features_by_domain_no_metadata(
            extraction.time_features_calc,
            filename,
            parts=parts
        )
        frame = pd.concat([frame, df])
    return frame


def features_frequency_domain(root: str, parts: int = PARTS) -> pd.DataFrame:
    frame = pd.DataFrame()
    for filename in extraction.fs_list_files(root):
        df = pumps.features_by_domain_no_metadata(
            extraction.frequency_features_calc,
            filename,
            window=FFT_WINDOW,
            parts=parts
        )
        frame = pd.concat([frame, df])
    return frame


def plot_features_boxplot(df: pd.DataFrame):
    fig, ax = plt.subplots(1, len(df.columns), figsize=(20, 4))
    for i, col in enumerate(df):
        df.boxplot([col], ax=ax[i])
    fig.tight_layout()
    plt.show()

In [ ]:
df = features_time_domain(PATH)
plot_features_boxplot(df)

In [ ]:
df = features_frequency_domain(PATH)
plot_features_boxplot(df)